# IOT HW1 - partC (a)

In [1]:
import os
import numpy as np
import pandas as pd

## 合併csv，並標記類別

In [2]:
# 到檔案資料夾內
try:
    os.chdir('./Data_Acc')
except:
    pass
print(os.getcwd())

# 合併csv
data = []
for root, dirs, files in os.walk('./'):
    print(root)
    print(dirs)
    print(files)
    for file in files:
        try:
            if(file != '.DS_Store'):
                temp = pd.read_csv(file, usecols = ['X','Y','Z'])
                if(file[0].upper() == 'R'):
                    label = 'Ride'
                elif(file[0].upper() == 'W'):
                    label = 'Walk'
                elif(file[0].upper() == 'S'):
                    label = 'Study'
                else:
                    label = 'X'
                temp['label'] = label
                data.append(temp)
        except:
            pass
            
# print(data)
result = pd.concat(data, ignore_index = True)
result.to_csv('result.csv', sep='\t', encoding='utf-8')

/Users/shungfu/IOT_HW_motiondata/partC/IOT partC(a)/Data_Acc
./
[]
['.DS_Store', 'Walk_Acc.csv', 'Study2_Acc.csv', 'result.csv', 'Riding2_Acc.csv', 'Study1_Acc.csv', 'Riding1_Acc.csv']


In [3]:
result

,X,Y,Z,label
0,0.256187,2.966506,9.409500,Walk
1,-2.669617,4.477294,9.488511,Walk
2,1.357554,-0.193936,9.093455,Walk
3,0.095771,0.088588,9.893144,Walk
4,0.114925,0.088588,9.890749,Walk
...,...,...,...,...
28540,-0.052674,0.098165,9.921875,Ride
28541,-0.011971,0.088588,9.921875,Ride
28542,-0.052674,0.098165,9.941029,Ride
28543,-0.035914,0.105348,9.919480,Ride


## label split

In [4]:
Data = result.iloc[:, :-1]
Target = pd.DataFrame(result.iloc[:, -1])

## label 轉為類別，字串轉數字


In [5]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

# label encoder
encoder = LabelEncoder()
encoder_y = encoder.fit_transform(Target)
# to category
target = pd.DataFrame(np_utils.to_categorical(encoder_y))
target

Using TensorFlow backend.
/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,0,1,2
0,0.0,0.0,1.0
1,0.0,0.0,1.0
2,0.0,0.0,1.0
3,0.0,0.0,1.0
4,0.0,0.0,1.0
...,...,...,...
28540,1.0,0.0,0.0
28541,1.0,0.0,0.0
28542,1.0,0.0,0.0
28543,1.0,0.0,0.0


## Sliding Window
window size = 100

In [6]:
# sliding window
window_size = 100

data = Data.rolling(window_size).mean()
# mask: data is nan
mask = data.isnull().any(axis=1)

# drop Nan
data = data.dropna()
target = target.drop(target[mask].index)

## Split test and train and shuffle

In [7]:
from sklearn.model_selection import train_test_split

data_train,data_test,target_train,target_test = train_test_split(data, target,test_size = 0.3)
print('Train Data Shape: ',data_train.shape, '\nTrain target Shape: ',target_train.shape)
print('Test Data Shape: ',data_test.shape, '\nTest target Shape: ',target_test.shape)

Train Data Shape:  (19912, 3) 
Train target Shape:  (19912, 3)
Test Data Shape:  (8534, 3) 
Test target Shape:  (8534, 3)


In [8]:
data_train

,X,Y,Z
13634,-0.210936,8.462564,5.894968
28041,0.565576,8.184948,5.155640
4883,-0.044629,0.044126,9.908634
2480,0.418998,3.296917,8.743892
16605,0.084805,-0.085811,9.881483
...,...,...,...
12333,2.825244,4.625739,8.037580
12890,0.356316,9.419580,-0.304264
17503,0.081405,-0.074223,9.902720
23337,0.081405,-0.074223,9.902720


In [9]:
target_train

,0,1,2
13634,1.0,0.0,0.0
28041,1.0,0.0,0.0
4883,0.0,0.0,1.0
2480,0.0,0.0,1.0
16605,0.0,1.0,0.0
...,...,...,...
12333,0.0,1.0,0.0
12890,1.0,0.0,0.0
17503,0.0,1.0,0.0
23337,0.0,1.0,0.0


## ANN models
using keras

In [10]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasClassifier

def ANN_model():
    # Using Relu activation, and Adam optimizer, 100 epochs
    model = Sequential([
        # input layer
        Dense(32,input_dim=3,activation='relu',kernel_initializer='uniform'),
        
        # hidden layer
        Dropout(0.2),
        Dense(16,activation='relu'),
        Dropout(0.2),
        
        # output layer
        Dense(3,activation='softmax'),
    ])

    model.summary()
#     model.add(Dense(32,input_dim=3,activation='relu',kernel_initializer='uniform'))

#     model.add(Dropout(0.2))
#     model.add(Dense(16,activation='relu'))
#     model.add(Dropout(0.2))

#     model.add(Dense(3,activation='softmax'))
    # Compile model
    model.compile(Adam(lr=.0001),loss = 'categorical_crossentropy', metrics=['accuracy'])
    return model

# Make it as sklearn model
estimator = KerasClassifier(build_fn=ANN_model,epochs= 100, batch_size=128,verbose=1)
# Train model
estimator.fit(data_train,target_train)


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 32)                128       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                528       
_________________________________________________________________
dropout_2 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 51        
Total params: 707
Trainable params: 707
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
19912/19912 [==============================] - 0s 24us/step - loss: 1.0524 - accuracy: 0.4709
Epoch 2/100
1991

19912/19912 [==============================] - 0s 10us/step - loss: 0.2272 - accuracy: 0.9447
Epoch 69/100
19912/19912 [==============================] - 0s 10us/step - loss: 0.2257 - accuracy: 0.9447
Epoch 70/100
19912/19912 [==============================] - 0s 9us/step - loss: 0.2253 - accuracy: 0.9444
Epoch 71/100
19912/19912 [==============================] - 0s 9us/step - loss: 0.2234 - accuracy: 0.9457
Epoch 72/100
19912/19912 [==============================] - 0s 9us/step - loss: 0.2225 - accuracy: 0.9450
Epoch 73/100
19912/19912 [==============================] - 0s 9us/step - loss: 0.2234 - accuracy: 0.9437
Epoch 74/100
19912/19912 [==============================] - 0s 9us/step - loss: 0.2212 - accuracy: 0.9462
Epoch 75/100
19912/19912 [==============================] - 0s 9us/step - loss: 0.2188 - accuracy: 0.9444
Epoch 76/100
19912/19912 [==============================] - 0s 9us/step - loss: 0.2186 - accuracy: 0.9442
Epoch 77/100
19912/19912 [==============================]

## Make prediction

In [11]:
pred = estimator.predict(data_test)
pred_label = encoder.inverse_transform(pred)

temp = target_test.copy().values
label = np.ones(shape=(len(temp),),dtype = int)
for i in range(len(label)):
    label[i] = np.argmax(temp[i])

print(label)
real_label = encoder.inverse_transform(label)

for i in range(100,120):
    print('Data[%d]'%(i),'Real: ', real_label[i],'Predict: ', pred_label[i], sep = '\n', end = '\n\n')

8534/8534 [==============================] - 0s 7us/step
[1 1 1 ... 2 2 1]
Data[100]
Real: 
Study
Predict: 
Study

Data[101]
Real: 
Study
Predict: 
Study

Data[102]
Real: 
Ride
Predict: 
Ride

Data[103]
Real: 
Ride
Predict: 
Ride

Data[104]
Real: 
Study
Predict: 
Study

Data[105]
Real: 
Study
Predict: 
Walk

Data[106]
Real: 
Study
Predict: 
Study

Data[107]
Real: 
Ride
Predict: 
Ride

Data[108]
Real: 
Study
Predict: 
Study

Data[109]
Real: 
Study
Predict: 
Study

Data[110]
Real: 
Study
Predict: 
Study

Data[111]
Real: 
Walk
Predict: 
Walk

Data[112]
Real: 
Ride
Predict: 
Ride

Data[113]
Real: 
Walk
Predict: 
Walk

Data[114]
Real: 
Study
Predict: 
Study

Data[115]
Real: 
Walk
Predict: 
Walk

Data[116]
Real: 
Study
Predict: 
Study

Data[117]
Real: 
Ride
Predict: 
Ride

Data[118]
Real: 
Walk
Predict: 
Walk

Data[119]
Real: 
Ride
Predict: 
Ride



## Eval Model

In [12]:
scores = estimator.score(data_test,target_test)
print('Test Accurancy: ',scores)

8534/8534 [==============================] - 0s 9us/step
Test Accurancy:  0.9641434550285339
